# Homework 2: training pipeline

This code will test your homework 2 solutions by using them in a complete ML pipeline. You should run this code in order to tune your model and save your model weights (which will also be uploaded as part of your solution)

In [19]:
# Download the training data from the homework2 folder:
# unzip using tar xzvvf nsynth_subset.tar.gz
# (this is a small subset of the "nsynth" dataset: https://magenta.tensorflow.org/datasets/nsynth)

### Setup

In [1]:
# !rm -rf nsynth_subset
!rm homework2.py
# !rm nsynth_subset.tar.gz

In [2]:
# Comment this out when submitting
# Nvm don't need to submit this file
# !tar -xvzf nsynth_subset.tar.gz

In [3]:
# Comment this out when submitting
# Nvm don't need to submit this file
from google.colab import files
uploaded = files.upload()

Saving homework2.py to homework2.py


In [4]:
import homework2

[-2.8059129e-03 -9.8028034e-03  1.2120705e-02 ... -3.2764859e-05
 -5.5099372e-06  8.2980841e-07]
1


### Install and Load Required Libraries  

In [5]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [7]:
BATCH_SIZE = 16
torch.use_deterministic_algorithms(True)

In [8]:
if not len(homework2.audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [9]:
# Some helper functions. These are the same as what the autograder runs.

In [10]:
# Split dataset into train / valid / test
def split_data(waveforms, labels, train_ratio=0.7, valid_ratio=0.15):
    assert(train_ratio + valid_ratio < 1)
    test_ratio = 1 - (train_ratio + valid_ratio)
    N = len(waveforms)
    Ntrain = int(N * train_ratio)
    Nvalid = int(N * valid_ratio)
    Ntest = int(N * test_ratio)
    Wtrain = waveforms[:Ntrain]
    Wvalid = waveforms[Ntrain:Ntrain + Nvalid]
    Wtest = waveforms[Ntrain + Nvalid:]
    ytrain = labels[:Ntrain]
    yvalid = labels[Ntrain:Ntrain + Nvalid]
    ytest = labels[Ntrain + Nvalid:]
    return Wtrain,Wvalid,Wtest,ytrain,yvalid,ytest

In [11]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [12]:
class InstrumentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        return features, torch.tensor(label, dtype=torch.long)

In [13]:
class Loaders():
    def __init__(self, waveforms, labels, feature_function, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        self.Wtrain, self.Wvalid, self.Wtest, self.ytrain, self.yvalid, self.ytest = split_data(waveforms, labels)

        self.Xtrain = process_data(self.Wtrain, feature_function)
        self.Xvalid = process_data(self.Wvalid, feature_function)
        self.Xtest = process_data(self.Wtest, feature_function)

        self.dataTrain = InstrumentDataset(self.Xtrain, self.ytrain)
        self.dataValid = InstrumentDataset(self.Xvalid, self.yvalid)
        self.dataTest = InstrumentDataset(self.Xtest, self.ytest)

        self.loaderTrain = DataLoader(self.dataTrain, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderValid = DataLoader(self.dataValid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(self.dataTest, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [14]:
class Pipeline():
    def __init__(self, module, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.criterion = nn.CrossEntropyLoss()

        self.model = module.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def evaluate(self, loader, which = "valid"):
        self.model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                #loss = criterion(outputs, labels) # validation loss

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total

        return acc

    def train(self, loaders,
          num_epochs=1, # Train for a single epoch by default
          model_path=None): # (Optionally) provide a path to save the best model
        val_acc = 0
        best_val_acc = 0
        for epoch in range(num_epochs):
            self.model.train()

            losses = []

            for inputs, labels in loaders.loaderTrain:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                losses.append(float(loss))

            self.model.eval()
            val_acc = self.evaluate(loaders.loaderValid)
            print("Epoch " + str(epoch) + ", loss = " + str(sum(losses)/len(losses)) +\
                  ", validation accuracy = " + str(val_acc))

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                if (model_path):
                    torch.save(self.model.state_dict(), model_path)
        print("Final validation accuracy = " + str(val_acc) + ", best = " + str(best_val_acc))
        return val_acc, best_val_acc

    def load(self, path):
        self.model.load_state_dict(torch.load(path, weights_only=True))

In [15]:
# The function below is the basis of how the autograder tests your code. Try to understand this one.

In [16]:
def test(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)

    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 10, path)

    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

### 1. Paths, labels, waveforms

In [ ]:
# 1. Paths, labels, waveforms

In [ ]:
# Once you've written the corresponding code in homework2.py, print these out or visualize them if you want
homework2.waveforms
homework2.labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,


### 2. MFCC

In [ ]:
# 2. MFCC

In [ ]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mfcc,
     homework2.MLPClassifier(),
     0.0001,
     "best_mlp_model.weights")

Extracting features...
Epoch 0, loss = 2.0775827964146933, validation accuracy = 0.6016260162601627
Epoch 1, loss = 0.8816034884916412, validation accuracy = 0.7479674796747967
Epoch 2, loss = 0.39115521642896867, validation accuracy = 0.8943089430894309
Epoch 3, loss = 0.2329025293389956, validation accuracy = 0.9349593495934959
Epoch 4, loss = 0.17220538419981798, validation accuracy = 0.943089430894309
Epoch 5, loss = 0.13788234235511887, validation accuracy = 0.959349593495935
Epoch 6, loss = 0.11388241706622972, validation accuracy = 0.975609756097561
Epoch 7, loss = 0.09591211316486199, validation accuracy = 0.983739837398374
Epoch 8, loss = 0.08206478702939218, validation accuracy = 0.991869918699187
Epoch 9, loss = 0.07096693685485257, validation accuracy = 0.991869918699187
Final validation accuracy = 0.991869918699187, best = 0.991869918699187
Test accuracy = 0.9919354838709677


### 3. Spectrogram

In [ ]:
# 3. Spectrogram

In [ ]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_spec,
     homework2.SimpleCNN(),
     0.0001,
     "best_spec_model.weights")

Extracting features...
Epoch 0, loss = 0.6056822074784173, validation accuracy = 0.9024390243902439
Epoch 1, loss = 0.5365122374561098, validation accuracy = 0.8861788617886179
Epoch 2, loss = 0.5077615189883444, validation accuracy = 0.8861788617886179
Epoch 3, loss = 0.4791342682308621, validation accuracy = 0.9024390243902439
Epoch 4, loss = 0.44440630823373795, validation accuracy = 0.8861788617886179
Epoch 5, loss = 0.39587850206428105, validation accuracy = 0.9024390243902439
Epoch 6, loss = 0.35085559967491364, validation accuracy = 0.9186991869918699
Epoch 7, loss = 0.3207702458732658, validation accuracy = 0.9349593495934959
Epoch 8, loss = 0.29952434326211613, validation accuracy = 0.967479674796748
Epoch 9, loss = 0.28269847979148227, validation accuracy = 0.967479674796748
Final validation accuracy = 0.967479674796748, best = 0.967479674796748
Test accuracy = 0.9354838709677419


### 4. Mel-spectrogram

In [ ]:
# 4. Mel-spectrogram

In [ ]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mel,
     homework2.SimpleCNN(),
     0.0001,
     "best_mel_model.weights")

Extracting features...
Epoch 0, loss = 0.47821613152821857, validation accuracy = 0.8048780487804879
Epoch 1, loss = 0.3436052319076326, validation accuracy = 0.8943089430894309
Epoch 2, loss = 0.2873547462125619, validation accuracy = 0.9349593495934959
Epoch 3, loss = 0.24040557733840412, validation accuracy = 0.9512195121951219
Epoch 4, loss = 0.20385145561562645, validation accuracy = 0.959349593495935
Epoch 5, loss = 0.17567643440432018, validation accuracy = 0.967479674796748
Epoch 6, loss = 0.1535542861868938, validation accuracy = 0.975609756097561
Epoch 7, loss = 0.13621011790302065, validation accuracy = 0.975609756097561
Epoch 8, loss = 0.1219639132420222, validation accuracy = 0.975609756097561
Epoch 9, loss = 0.10995618874828021, validation accuracy = 0.983739837398374
Final validation accuracy = 0.983739837398374, best = 0.983739837398374
Test accuracy = 1.0


### 5. Constant-Q

In [16]:
# 5. Constant-Q

In [17]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_q_model.weights")

Extracting features...
Epoch 0, loss = 0.5864689490861363, validation accuracy = 0.7560975609756098
Epoch 1, loss = 0.4836585306459003, validation accuracy = 0.7967479674796748
Epoch 2, loss = 0.4185015343957477, validation accuracy = 0.8211382113821138
Epoch 3, loss = 0.3658535120387872, validation accuracy = 0.8536585365853658
Epoch 4, loss = 0.32844820370276767, validation accuracy = 0.9024390243902439
Epoch 5, loss = 0.29572899474038017, validation accuracy = 0.9105691056910569
Epoch 6, loss = 0.2667641006410122, validation accuracy = 0.9186991869918699
Epoch 7, loss = 0.24122309808929762, validation accuracy = 0.9186991869918699
Epoch 8, loss = 0.2188968857129415, validation accuracy = 0.9349593495934959
Epoch 9, loss = 0.19927269178960058, validation accuracy = 0.9349593495934959
Final validation accuracy = 0.9349593495934959, best = 0.9349593495934959
Test accuracy = 0.9435483870967742


### 6. Pitch shift

In [18]:
# 6. Pitch shift

In [19]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [20]:
test(homework2.augmented_waveforms,
     homework2.augmented_labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_augmented_model.weights")

Extracting features...
Epoch 0, loss = 0.5219301860089656, validation accuracy = 0.8157181571815718
Epoch 1, loss = 0.386605063798251, validation accuracy = 0.9186991869918699
Epoch 2, loss = 0.32611309585196, validation accuracy = 0.959349593495935
Epoch 3, loss = 0.2855228246362121, validation accuracy = 0.9728997289972899
Epoch 4, loss = 0.253652014873094, validation accuracy = 0.975609756097561
Epoch 5, loss = 0.22662092234801362, validation accuracy = 0.978319783197832
Epoch 6, loss = 0.2032376746474593, validation accuracy = 0.981029810298103
Epoch 7, loss = 0.18260153500294243, validation accuracy = 0.983739837398374
Epoch 8, loss = 0.1643435505943166, validation accuracy = 0.983739837398374
Epoch 9, loss = 0.14811173184878296, validation accuracy = 0.986449864498645
Final validation accuracy = 0.986449864498645, best = 0.986449864498645
Test accuracy = 0.9756756756756757


### 7. Extend your model to handle four classes and creatively improve its performance

In [19]:
# 7. Extend your model to handle four classes and creatively improve its performance

In [20]:
test(homework2.waveforms,
     homework2.labels_7,
     homework2.feature_func_7,
     homework2.model_7,
     0.0001,
     "best_model_7.weights")

Extracting features...
Epoch 0, loss = 0.49716028778089416, validation accuracy = 0.8699186991869918
Epoch 1, loss = 0.44990811290012467, validation accuracy = 0.8780487804878049
Epoch 2, loss = 0.4171382031506962, validation accuracy = 0.8861788617886179
Epoch 3, loss = 0.39176949817273354, validation accuracy = 0.9024390243902439
Epoch 4, loss = 0.3705664235684607, validation accuracy = 0.9024390243902439
Epoch 5, loss = 0.35342773670951527, validation accuracy = 0.9105691056910569
Epoch 6, loss = 0.3388783811695046, validation accuracy = 0.9105691056910569
Epoch 7, loss = 0.32626512874331737, validation accuracy = 0.9186991869918699
Epoch 8, loss = 0.31453477177355027, validation accuracy = 0.9186991869918699
Epoch 9, loss = 0.3044412506537305, validation accuracy = 0.9186991869918699
Final validation accuracy = 0.9186991869918699, best = 0.9186991869918699
Test accuracy = 0.8548387096774194
